# Importing Libraries

In [11]:
import torch
import torchvision
from torchvision import transforms
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset
Resnet requires the input size to be (224,224). So, they are resized.

In [13]:
path = '/kaggle/input/flowers-recognition/flowers'

transfrom = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# Train Test Split
Train = 80%   Test = 20%

In [14]:
dataset = ImageFolder(path,transform=transfrom)
train, val = train_test_split(dataset,test_size=0.2,shuffle=True,random_state=7)
 
train_loader = DataLoader(train, batch_size=32,shuffle=True)
val_loader = DataLoader(val, batch_size=32)

# Model
Using the base model from the ResNet model that is pre-trained on the ImageNet dataset.

In [15]:
model = torchvision.models.resnet18(pretrained=True).to(device)

Freezing the convolutional base and using it as a feature extractor. It makes the weight non-trainable. So, its value is no longer updated during training.

In [16]:
for parameter in model.parameters():
    parameter.requires_grad = False

The last fully connected layer is replaced with a new one with random weights and only this layer is trained. Finally, reshaping the final layer to have the same number of outputs as the number of classes in the new dataset.

In [17]:
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, len(dataset.classes)).to(device)
loss_func = torch.nn.CrossEntropyLoss().to(device)

# Optimizer

In [18]:
params_to_update = []
for name, parameter in model.named_parameters():
    if parameter.requires_grad == True:
        params_to_update.append(parameter)

optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Train And Test

In [19]:
def train(model,loss_func,dataloader,optimizer,epoch):
  print('\nEpoch : %d'%epoch)
  
  total_loss=0    
  correct=0
  total=0

  model.train()

  for data in tqdm(dataloader):
    
    inputs,labels=data[0].to(device),data[1].to(device)
    outputs=model(inputs)
    
    loss=loss_func(outputs,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
      
  loss=total_loss/len(dataloader)
  acc=100.*correct/total

  print('Train Loss: %.3f | Accuracy: %.3f'%(loss,acc))

In [20]:
def test(model,loss_func,dataloader,epoch):
  model.eval()

  total_loss=0
  correct=0
  total=0

  with torch.no_grad():
    for data in tqdm(dataloader):
      images,labels=data[0].to(device),data[1].to(device)
      outputs=model(images)

      loss= loss_func(outputs,labels)
      total_loss+=loss.item()
      
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()
  
  loss=total_loss/len(dataloader)
  acc=100.*correct/total

  print('Test Loss: %.3f | Accuracy: %.3f'%(loss,acc)) 


In [21]:
epochs = 20
for epoch in range(1, epochs+1):
    train(model, loss_func, train_loader, optimizer, epoch)
    test(model, loss_func, val_loader, epoch)


Epoch : 1


100%|██████████| 108/108 [00:02<00:00, 37.68it/s]


Train Loss: 0.984 | Accuracy: 64.466


100%|██████████| 27/27 [00:00<00:00, 41.30it/s]


Test Loss: 0.694 | Accuracy: 77.199

Epoch : 2


...

Test Loss: 0.421 | Accuracy: 84.954

Epoch : 20


100%|██████████| 108/108 [00:02<00:00, 41.86it/s]


Train Loss: 0.319 | Accuracy: 89.198


100%|██████████| 27/27 [00:00<00:00, 42.59it/s]

Test Loss: 0.418 | Accuracy: 84.838
